In [1]:
import torch  # This is all you need to use both PyTorch and TorchScript!
print(torch.__version__)
torch.manual_seed(191009)  # set the seed for reproducibility

2.4.0.dev20240414+cu118


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class MyModule(nn.Module):
    def __init__(self):
        super().__init__()
        # torch.jit.trace produces a ScriptModule's conv1 and conv2
        self.conv1 = torch.jit.trace(nn.Conv2d(1, 20, 5), torch.rand(1, 1, 16, 16))
        self.conv2 = torch.jit.trace(nn.Conv2d(20, 20, 5), torch.rand(1, 20, 16, 16))

    def forward(self, input):
        input = F.relu(self.conv1(input))
        input = F.relu(self.conv2(input))
        return input

scripted_module = torch.jit.script(MyModule())
print(scripted_module)


RecursiveScriptModule(
  original_name=MyModule
  (conv1): Conv2d(original_name=Conv2d)
  (conv2): Conv2d(original_name=Conv2d)
)


In [3]:
class MyCell(torch.nn.Module):
    def __init__(self):
        super(MyCell, self).__init__()
        self.linear = torch.nn.Linear(4, 4)

    def forward(self, x, h):
        new_h = torch.tanh(self.linear(x) + h)
        return new_h, new_h

my_cell = MyCell()
x, h = torch.rand(3, 4), torch.rand(3, 4)
traced_cell = torch.jit.trace(my_cell, (x, h))
print(traced_cell)
traced_cell(x, h)




MyCell(
  original_name=MyCell
  (linear): Linear(original_name=Linear)
)


(tensor([[-0.1744,  0.3766,  0.6543,  0.6161],
         [-0.3007,  0.2489,  0.5094,  0.3352],
         [-0.4256,  0.9030,  0.5932,  0.6982]], grad_fn=<TanhBackward0>),
 tensor([[-0.1744,  0.3766,  0.6543,  0.6161],
         [-0.3007,  0.2489,  0.5094,  0.3352],
         [-0.4256,  0.9030,  0.5932,  0.6982]], grad_fn=<TanhBackward0>))

In [4]:
print(traced_cell.graph)

graph(%self.1 : __torch__.MyCell,
      %x : Float(3, 4, strides=[4, 1], requires_grad=0, device=cpu),
      %h : Float(3, 4, strides=[4, 1], requires_grad=0, device=cpu)):
  %linear : __torch__.torch.nn.modules.linear.Linear = prim::GetAttr[name="linear"](%self.1)
  %20 : Tensor = prim::CallMethod[name="forward"](%linear, %x)
  %11 : int = prim::Constant[value=1]() # C:\Users\zuolu\AppData\Local\Temp\ipykernel_27000\723592951.py:7:0
  %12 : Float(3, 4, strides=[4, 1], requires_grad=1, device=cpu) = aten::add(%20, %h, %11) # C:\Users\zuolu\AppData\Local\Temp\ipykernel_27000\723592951.py:7:0
  %13 : Float(3, 4, strides=[4, 1], requires_grad=1, device=cpu) = aten::tanh(%12) # C:\Users\zuolu\AppData\Local\Temp\ipykernel_27000\723592951.py:7:0
  %14 : (Float(3, 4, strides=[4, 1], requires_grad=1, device=cpu), Float(3, 4, strides=[4, 1], requires_grad=1, device=cpu)) = prim::TupleConstruct(%13, %13)
  return (%14)



In [5]:
print(my_cell(x, h))
print(traced_cell(x, h))

(tensor([[-0.1744,  0.3766,  0.6543,  0.6161],
        [-0.3007,  0.2489,  0.5094,  0.3352],
        [-0.4256,  0.9030,  0.5932,  0.6982]], grad_fn=<TanhBackward0>), tensor([[-0.1744,  0.3766,  0.6543,  0.6161],
        [-0.3007,  0.2489,  0.5094,  0.3352],
        [-0.4256,  0.9030,  0.5932,  0.6982]], grad_fn=<TanhBackward0>))
(tensor([[-0.1744,  0.3766,  0.6543,  0.6161],
        [-0.3007,  0.2489,  0.5094,  0.3352],
        [-0.4256,  0.9030,  0.5932,  0.6982]], grad_fn=<TanhBackward0>), tensor([[-0.1744,  0.3766,  0.6543,  0.6161],
        [-0.3007,  0.2489,  0.5094,  0.3352],
        [-0.4256,  0.9030,  0.5932,  0.6982]], grad_fn=<TanhBackward0>))


In [6]:
class MyDecisionGate(torch.nn.Module):
    def forward(self, x):
        if x.sum() > 0:
            return x
        else:
            return -x

class MyCell(torch.nn.Module):
    def __init__(self, dg):
        super(MyCell, self).__init__()
        self.dg = dg
        self.linear = torch.nn.Linear(4, 4)

    def forward(self, x, h):
        new_h = torch.tanh(self.dg(self.linear(x)) + h)
        return new_h, new_h

my_cell = MyCell(MyDecisionGate())
traced_cell = torch.jit.trace(my_cell, (x, h))

print(traced_cell.dg.code)
print(traced_cell.code)

def forward(self,
    argument_1: Tensor) -> NoneType:
  return None

def forward(self,
    x: Tensor,
    h: Tensor) -> Tuple[Tensor, Tensor]:
  dg = self.dg
  linear = self.linear
  _0 = (linear).forward(x, )
  _1 = (dg).forward(_0, )
  _2 = torch.tanh(torch.add(_0, h))
  return (_2, _2)



C:\Users\zuolu\AppData\Local\Temp\ipykernel_27000\4234398751.py:3: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if x.sum() > 0:


In [7]:
scripted_gate = torch.jit.script(MyDecisionGate())

my_cell = MyCell(scripted_gate)
scripted_cell = torch.jit.script(my_cell)

print(scripted_gate.code)
print(scripted_cell.code)

def forward(self,
    x: Tensor) -> Tensor:
  if bool(torch.gt(torch.sum(x), 0)):
    _0 = x
  else:
    _0 = torch.neg(x)
  return _0

def forward(self,
    x: Tensor,
    h: Tensor) -> Tuple[Tensor, Tensor]:
  dg = self.dg
  linear = self.linear
  _0 = torch.add((dg).forward((linear).forward(x, ), ), h)
  new_h = torch.tanh(_0)
  return (new_h, new_h)

